# Selecting samples for Synthetic data generation

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os
import sys
from pathlib import Path

path = Path(os.path.dirname(os.getcwd()))
path = str(path)
print(path)
sys.path.insert(1, path)

/Users/saideepbunny/Projects/Application_Ranking_System


## Reading the data

In [3]:
df = pd.read_csv(f"{path}/data/kaggle_linkedin_jd_skills_dataset/merged_selected_jd.csv")
df

,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_skills,job_summary,id,summary_len
0,https://www.linkedin.com/jobs/view/independent...,2024-01-21 07:40:00.308133+00,t,t,f,Independent Real Estate Agent,Howard Hanna | Rand Realty,"Englewood Cliffs, NJ",2024-01-16,Pinehurst,United States,Real-Estate Clerk,Mid senior,Onsite,"Real Estate, Customer Service, Sales, Negotiat...",Who We Are\nRand Realty is a family-owned brok...,3,3944
1,https://www.linkedin.com/jobs/view/chef-de-cui...,2024-01-21 07:12:35.170823+00,t,t,f,Chef de Cuisine,Goodwin Recruiting,"Brookfield, WI",2024-01-14,Milwaukee,United States,Sous Chef,Mid senior,Onsite,"Chef de Cuisine, Executive Sous Chef, Hospital...",To Apply for this Job Click Here\nMidwest Hosp...,20,2789
2,https://www.linkedin.com/jobs/view/cheese-spec...,2024-01-21 06:01:14.692423+00,t,t,f,Cheese Specialist,Safeway,"Mount Airy, MD",2024-01-14,Frederick,United States,Special Effects Specialist,Mid senior,Onsite,"Retail, Grocery, Customer service, Merchandisi...",CHEESE SPECIALIST\nWhy you will choose us:\nAl...,30,3714
3,https://www.linkedin.com/jobs/view/retail-dist...,2024-01-21 07:12:35.189652+00,t,t,f,RETAIL DISTRICT MANAGER UNASSIGNED,Dollar General,"Newport, NH",2024-01-14,New Hampshire,United States,Pressure Supervisor,Mid senior,Onsite,"Retail Management, Profit and Loss Management,...",Dollar General Corporation has been delivering...,34,4372
4,https://www.linkedin.com/jobs/view/travel-rn-l...,2024-01-21 07:12:37.537623+00,t,t,f,Travel RN - Long Term Care,CoreMedical Group,"Barnstable, MA",2024-01-14,Barnstable,United States,Nurse Head,Mid senior,Onsite,"Travel Nursing, Long Term Care, RN (Registered...",Core Medical Group is seeking a travel nurse R...,40,3163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125375,https://uk.linkedin.com/jobs/view/production-p...,2024-01-20 04:01:08.913697+00,t,t,f,Production Planner,Eat Real + PROPER | WARP Snacks,"Nuneaton, England, United Kingdom",2024-01-14,Hastings,United Kingdom,Scheduler,Mid senior,Onsite,"Production Planning, ERP (Access), Scheduling,...",About WARP\nPROPER and Eat Real were born out ...,1296269,2614
125376,https://www.linkedin.com/jobs/view/store-manag...,2024-01-20 05:12:47.255315+00,t,t,f,Store Manager in Training,O'Reilly Auto Parts,"Concord, NH",2024-01-14,Laconia,United States,Bakery Supervisor,Mid senior,Onsite,"Sales, Leadership, Hiring, Training, Safety, C...","Assist store manager in areas of sales, appear...",1296275,4067
125377,https://www.linkedin.com/jobs/view/sales-repre...,2024-01-20 13:46:06.129745+00,t,t,f,Sales Representative,American Fidelity Sales Careers,"Altoona, PA",2024-01-14,Pennsylvania,United States,Group-Sales Representative,Associate,Onsite,"Sales, Insurance, Account Management, Business...",American Fidelity Assurance is now looking for...,1296347,3218
125378,https://www.linkedin.com/jobs/view/ft-center-s...,2024-01-20 13:46:10.289403+00,t,t,f,FT Center Store Manager (H),Food Lion,"Powhatan, VA",2024-01-14,Abingdon,United States,Manager Employment,Mid senior,Onsite,"Grocery Department Management, Hiring and Trai...",Address:\nUSA-VA-Powhatan-1800 South Creek One...,1296358,5770


In [4]:
df.isna().sum()

job_link                 0
last_processed_time      0
got_summary              0
got_ner                  0
is_being_worked          0
job_title                0
company                  3
job_location             2
first_seen               0
search_city              0
search_country           0
search_position          0
job_level                0
job_type                 0
job_skills             216
job_summary              0
id                       0
summary_len              0
dtype: int64

In [5]:
#drop those records with null values in job_skills column
df = df.dropna(subset=['job_skills']).copy()

## Examining job summaries

### All job summaries

In [6]:
print(df.loc[0]['job_summary'])

Who We Are
Rand Realty is a family-owned brokerage born in 1984 when our founder, Marsha opened her new real estate office in a small town in Hudson Valley. Her vision was to build a brokerage strongly devoted to its agents and deeply committed to the community combining expertise and eye for detail with heart, soul and kindness. More than 35 years later, we've grown to more than 1,000 premier local agents with offices throughout Northern New Jersey, Westchester, Connecticut, The Bronx, Rockland, Dutchess, Putnam and Orange counties. Despite our growth or really, by virtue of it and the great people who have joined and represent our brand, the characteristics that defined us then, continue to dictate and delineate who we are.
Today, Howard Hanna | Rand Realty is the preeminent real estate brokerage in New York, New Jersey and Connecticut, widely recognized as an industry leader for its agent training, career planning, technological innovation and market expertise. Build your career wit

In [7]:
print(df.loc[1]['job_summary'])

To Apply for this Job Click Here
Midwest Hospitality Group seeks a Chef de Cuisine to lead its Culinary Team. The ideal candidate will possess experience as a Chef de Cuisine and/or Executive Sous Chef in Independent Restaurants or Hotel Food & Beverage operations. Are you looking for a new challenge with a stable Company whose Culture is focused on providing the Guest with the highest quality Food and Service and offers growth opportunities? If so, please submit your resume today!
Benefits
Salary - $80,000 to $90,000
Health insurance
Life insurance
Free Parking
Shift Meals
PTO immediately
Company Discounts
Responsibilities
Ensure proper team coverage to execute a high level of production while keeping payroll costs in line
Oversee ordering, receiving, storage (including temperature-setting) usage, and rotation of food
Works with Maintenance in the repair of equipment and preventative maintenance programs
Assists with menu development and engineering
Leads culinary staff in the proper 

We can see from the above sample that few of the job summaries specifically have "Responsibilities" in the text. Since it is not possible to evaluate all the ~125k records to see if the job summary includes Responsibilities, we can simply filter the records based on the existence of the string "Responsibilities".

### Records containing "Responsibilities" in job summary

In [8]:
resp_df = df[df['job_summary'].str.contains("Responsibilities")]
resp_df

,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_skills,job_summary,id,summary_len
1,https://www.linkedin.com/jobs/view/chef-de-cui...,2024-01-21 07:12:35.170823+00,t,t,f,Chef de Cuisine,Goodwin Recruiting,"Brookfield, WI",2024-01-14,Milwaukee,United States,Sous Chef,Mid senior,Onsite,"Chef de Cuisine, Executive Sous Chef, Hospital...",To Apply for this Job Click Here\nMidwest Hosp...,20,2789
7,https://www.linkedin.com/jobs/view/rad-tech-at...,2024-01-19 14:39:33.898387+00,t,t,f,Rad Tech,Virginia Mason Franciscan Health,"Seattle, WA",2024-01-13,Summit,United States,Radiologist,Mid senior,Onsite,"Radiology, Diagnostic imaging, Xray, Fluorosco...","Overview\nUp to a $10,000 sign on bonus**\nVir...",79,2506
8,https://www.linkedin.com/jobs/view/retail-stoc...,2024-01-21 07:43:49.202444+00,t,t,f,Retail Stocking Team Lead,"Burlington Stores, Inc.","Chandler, AZ",2024-01-16,Chandler,United States,Pattern-Shop Supervisor,Mid senior,Onsite,"Retail stocking, Leadership, Problem solving, ...",LOCATION\n2630 E Germann Rd Chandler AZ US 852...,85,3009
9,https://www.linkedin.com/jobs/view/grooming-sa...,2024-01-21 07:43:50.084041+00,t,t,f,Grooming Salon Manager,Petco,"Sarasota, FL",2024-01-16,Sarasota,United States,Quality Assurance Monitor,Mid senior,Onsite,"Pet Grooming, Salon Management, Leadership, Cu...","Create a healthier, brighter future for pets, ...",92,5625
10,https://ca.linkedin.com/jobs/view/senior-produ...,2024-01-21 07:43:52.439593+00,t,t,f,Senior Product Manager,Robert Half,"Calgary, Alberta, Canada",2024-01-14,Calgary,Canada,Tier,Mid senior,Onsite,"Product Management, P&L Management, Sales Acum...",Description\nThe Role\nJoin our client's dynam...,97,4454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125374,https://www.linkedin.com/jobs/view/quality-aud...,2024-01-20 04:01:06.622679+00,t,t,f,Quality Auditor,ANGARAI,"Washington, DC",2024-01-14,Washington,United States,Standards Engineer,Mid senior,Onsite,"Project Management, Quality Audits, Quality Co...",Job Title: Quality Auditor\nJob Location:\nWas...,1296268,2288
125375,https://uk.linkedin.com/jobs/view/production-p...,2024-01-20 04:01:08.913697+00,t,t,f,Production Planner,Eat Real + PROPER | WARP Snacks,"Nuneaton, England, United Kingdom",2024-01-14,Hastings,United Kingdom,Scheduler,Mid senior,Onsite,"Production Planning, ERP (Access), Scheduling,...",About WARP\nPROPER and Eat Real were born out ...,1296269,2614
125377,https://www.linkedin.com/jobs/view/sales-repre...,2024-01-20 13:46:06.129745+00,t,t,f,Sales Representative,American Fidelity Sales Careers,"Altoona, PA",2024-01-14,Pennsylvania,United States,Group-Sales Representative,Associate,Onsite,"Sales, Insurance, Account Management, Business...",American Fidelity Assurance is now looking for...,1296347,3218
125378,https://www.linkedin.com/jobs/view/ft-center-s...,2024-01-20 13:46:10.289403+00,t,t,f,FT Center Store Manager (H),Food Lion,"Powhatan, VA",2024-01-14,Abingdon,United States,Manager Employment,Mid senior,Onsite,"Grocery Department Management, Hiring and Trai...",Address:\nUSA-VA-Powhatan-1800 South Creek One...,1296358,5770


In [9]:
for i in range(10):
    print(f"Record {i+1}:")
    print(resp_df.iloc[i]['job_summary'])
    print("\n================================================================================\n")

Record 1:
To Apply for this Job Click Here
Midwest Hospitality Group seeks a Chef de Cuisine to lead its Culinary Team. The ideal candidate will possess experience as a Chef de Cuisine and/or Executive Sous Chef in Independent Restaurants or Hotel Food & Beverage operations. Are you looking for a new challenge with a stable Company whose Culture is focused on providing the Guest with the highest quality Food and Service and offers growth opportunities? If so, please submit your resume today!
Benefits
Salary - $80,000 to $90,000
Health insurance
Life insurance
Free Parking
Shift Meals
PTO immediately
Company Discounts
Responsibilities
Ensure proper team coverage to execute a high level of production while keeping payroll costs in line
Oversee ordering, receiving, storage (including temperature-setting) usage, and rotation of food
Works with Maintenance in the repair of equipment and preventative maintenance programs
Assists with menu development and engineering
Leads culinary staff in t

## Statistics for summary length and skill count

In [10]:
#Splitting job skills by comma to get skill count
resp_df.loc[:,'skill_count'] = resp_df['job_skills'].apply(lambda x: len(x.split(","))).copy()


/var/folders/t5/tflj5hjs197_vv37fw2sdhdr0000gn/T/ipykernel_11981/3804758628.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resp_df.loc[:,'skill_count'] = resp_df['job_skills'].apply(lambda x: len(x.split(","))).copy()


In [11]:
#getting max summary_len for each job title using groupby
resp_df[['job_title', 'summary_len', 'skill_count']].groupby(['job_title']).aggregate({"summary_len":"max", "skill_count":"max"})

,summary_len,skill_count
job_title,,
2024 - Culinary Professionals - Assistant Kitchen Manager / Sous Chef,5863,69
2024 - Culinary Professionals - Dining Hall Service Manager,5593,69
AE - Sales Leader (Full-Time) - US,4997,46
ARNP,6175,56
ASSISTANT MANAGER,5375,99
...,...,...
Workshop Controller,4703,32
X-Ray Technologist,6213,41
Xfinity Retail Sales Consultant,4972,32


In [12]:
print("Number of unique job titles:", resp_df['job_title'].nunique())

Number of unique job titles: 1168


The plan is to obtain 1500 records for generating synthetic resumes. Therefore, it would be better to choose the job_summary that has the most information about the job. So, for each job title, I will select the record with max. summary_len. That makes 1168 records. For the rest of the records, I will select some famous job titles like Data Engineer, Data Scientist, etc., that have the highest number of skill_count

## Final Sampling

### Sample 1 Job per job title

In [13]:
#Getting job that has maximum length job_summary for each job title
temp = resp_df[resp_df['skill_count'] >=30].copy()
resp_df_max = temp.loc[temp.groupby('job_title')['summary_len'].idxmax()].reset_index(drop=True).copy()
resp_df_max = resp_df_max.drop_duplicates(keep='first', subset=['job_summary'])
resp_df_max

,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_skills,job_summary,id,summary_len,skill_count
0,https://www.linkedin.com/jobs/view/2024-culina...,2024-01-19 14:51:42.44518+00,t,t,f,2024 - Culinary Professionals - Assistant Kitc...,"Wolfoods, Inc.","Oconomowoc, WI",2024-01-14,Wisconsin,United States,Sous Chef,Mid senior,Onsite,"Culinary Skills, Scratch Food Production, Supe...",Wolfoods operates summer camp kitchens across ...,62736,5863,30
1,https://www.linkedin.com/jobs/view/2024-culina...,2024-01-19 12:35:58.423951+00,t,t,f,2024 - Culinary Professionals - Dining Hall Se...,"Wolfoods, Inc.","Anchorage, AK",2024-01-15,Alaska,United States,Cook Fast Food,Mid senior,Onsite,"Project Management, Control Inventory, Communi...",Wolfoods operates summer camp kitchens across ...,34791,5593,43
2,https://www.linkedin.com/jobs/view/ae-sales-le...,2024-01-19 09:45:09.215838+00,t,t,f,AE - Sales Leader (Full-Time) - US,AMERICAN EAGLE OUTFITTERS INC.,"Providence, RI",2024-01-12,Attleboro,United States,Account Executive,Mid senior,Onsite,"Sales Leadership, LOD (Leader on Duty), Guest ...",BE PART OF SOMETHING\nREAL\nYOUR ROLE\nAs the ...,623445,4994,30
3,https://www.linkedin.com/jobs/view/arnp-at-fas...,2024-01-20 11:08:01.254837+00,t,t,f,ARNP,Fast Pace Health,"Prien, LA",2024-01-14,Lake Charles,United States,Nurse Practitioner,Mid senior,Onsite,"Family Nurse Practitioner, Teamwork, Communica...",Posting Title: Family Nurse Practitioner Overv...,955798,6118,31
4,https://www.linkedin.com/jobs/view/assistant-m...,2024-01-20 03:22:05.147606+00,t,t,f,ASSISTANT MANAGER,d'Lite Healthy On The Go,"Scottsdale, AZ",2024-01-15,Chandler,United States,Complaint Clerk,Mid senior,Onsite,"Restaurant management, Leadership, Customer se...","Location: Scottsdale, AZ\nEmployment Type: Ful...",13071,5375,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,https://uk.linkedin.com/jobs/view/workshop-con...,2024-01-19 09:45:09.215838+00,t,t,f,Workshop Controller,Activate Group Limited,"Bury St. Edmunds, England, United Kingdom",2024-01-13,Ipswich,United Kingdom,Controller,Mid senior,Onsite,"Workshop Control, Body Shop Management, HighQu...",Job title: Workshop Controller\nDepartment: Ac...,321651,4703,32
1010,https://www.linkedin.com/jobs/view/x-ray-techn...,2024-01-21 04:16:54.473867+00,t,t,f,X-Ray Technologist,Fast Pace Health,"Grant, AL",2024-01-17,Guntersville,United States,Ultrasound Technologist,Mid senior,Onsite,"XRay Technologist, HIPAA, Patient intake, Nurs...",Posting Title: X-Ray Technologist Overview:\nF...,1230454,5704,38
1011,https://www.linkedin.com/jobs/view/xfinity-ret...,2024-01-19 23:17:04.320124+00,t,t,f,Xfinity Retail Sales Consultant,XFINITY Store by Comcast,"East Brunswick, NJ",2024-01-14,Long Branch,United States,Sales-Service Promoter,Mid senior,Onsite,"Customer service, Product demonstration, Probl...",Job Summary\nResponsible for assisting and con...,159258,4839,32
1012,https://www.linkedin.com/jobs/view/yard-associ...,2024-01-19 09:45:09.215838+00,t,t,f,Yard Associate (SkillBridge),ARMOR Initiative,"Kansas City, MO",2024-01-13,Leavenworth,United States,Orderly,Associate,Onsite,"DoD SkillBridge Internship, Fulltime training,...",The ARMOR Initiative DoD SkillBridge Internshi...,774643,2277,30


### Sampling remaining 339 Jobs

In [14]:
#Getting Series of unique job_titles
job_titles = resp_df['job_title'].value_counts().index

In [15]:
#Gathering other job titles
title_subset = job_titles[job_titles.str.contains("Data")].tolist() + job_titles[job_titles.str.contains("Software")].tolist() + job_titles[job_titles.str.contains("Research")].tolist() + job_titles[job_titles.str.contains("Generative")].tolist() + job_titles[job_titles.str.contains("System")].tolist() + job_titles[job_titles.str.contains("UX")].tolist()

len(title_subset)

50

In [16]:
# Filter the DataFrame first
filtered_resp_df = resp_df[(resp_df['job_title'].isin(title_subset)) & (resp_df['skill_count'] >= 30) & (~resp_df['id'].isin(resp_df_max['id']))]

# Check if you have at least 339 records to sample
if len(filtered_resp_df) >= 492:
    resp_df_sample = filtered_resp_df.sample(n=492, random_state=32)
else:
    resp_df_sample = filtered_resp_df  # or handle this case as required
    print(f"Warning: Only {len(resp_df_sample)} records available; fewer than 492.")

resp_df_sample

,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_skills,job_summary,id,summary_len,skill_count
34064,https://www.linkedin.com/jobs/view/software-ar...,2024-01-19 09:45:09.215838+00,t,t,f,Software Architect,Harris Computer,"Florida, United States",2024-01-13,Beverly Hills,United States,Director Technical,Mid senior,Onsite,"Product architecture, Engineering processes, C...",As Architect you will define the products’ hig...,340370,6407,33
63847,https://www.linkedin.com/jobs/view/senior-syst...,2024-01-19 09:45:09.215838+00,t,t,f,Senior System Engineer,ExLabs,"Long Beach, CA",2024-01-13,Compton,United States,Maintenance Data Analyst,Mid senior,Onsite,"Systems engineering, Technical oversight, Requ...",Job Title: Sr. System Engineer\nCompany: ExLab...,640026,6308,32
106562,https://www.linkedin.com/jobs/view/senior-data...,2024-01-20 11:15:59.804413+00,t,t,f,Senior Data Scientist,Workforce Connections,"St Louis, MO",2024-01-14,Belleville,United States,Chemist,Mid senior,Onsite,"Data Science, Data Analysis, Machine Learning,...","Location: Onsite, St. Louis MO 63146\nJob Resp...",1096469,3635,30
97378,https://www.linkedin.com/jobs/view/distinguish...,2024-01-19 22:25:24.801979+00,t,t,f,Distinguished Applied Researcher,Jobs for Humanity,"Fort Worth, TX",2024-01-15,Fort Worth,United States,Director Research,Mid senior,Onsite,"AI, Applied Research, Machine Learning, Natura...",Company Description\nJobs for Humanity is part...,998553,4011,34
115276,https://www.linkedin.com/jobs/view/systems-adm...,2024-01-21 01:06:09.90334+00,t,t,f,Systems Administrator,Leidos,"Colorado Springs, CO",2024-01-14,Cripple Creek,United States,Tier,Mid senior,Onsite,"System Administration, IT Operating Systems, C...",Description\nThe Leidos Digital Modernization ...,1189859,4117,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112773,https://www.linkedin.com/jobs/view/business-sy...,2024-01-20 16:33:23.92565+00,t,t,f,Business Systems Analyst,SPECTRAFORCE,"Seattle, WA",2024-01-14,Summit,United States,Systems Analyst,Mid senior,Onsite,"Business Systems Analyst, SuccessFactors Emplo...",Job Title- Business Systems Analyst\nDuration-...,1162353,3214,37
63885,https://www.linkedin.com/jobs/view/software-ar...,2024-01-19 09:45:09.215838+00,t,t,f,Software Architect,Software People Inc.,"Albany, NY",2024-01-13,Cohoes,United States,Guide,Mid senior,Onsite,"Systems Architecture, Enterprise Architecture,...",Client: New York State Office of the State Com...,640468,3501,40
52900,https://www.linkedin.com/jobs/view/principal-s...,2024-01-19 09:45:09.215838+00,t,t,f,Principal Software Engineer,Syrinx Consulting,"Bedford, MA",2024-01-13,Haverhill,United States,Computer Systems Hardware Analyst,Mid senior,Onsite,"Agile Development, Software Development, Techn...","In this role, you will work in an agile develo...",522773,2968,31
15817,https://uk.linkedin.com/jobs/view/lead-systems...,2024-01-19 23:23:21.954466+00,t,t,f,Lead Systems Engineer,Matchtech,"Farnborough, England, United Kingdom",2024-01-14,Reading,United Kingdom,Computer Systems Hardware Analyst,Mid senior,Onsite,"Security Clearance, Systems Engineering, Proje...",Overview\nImportant:\nAll applicants must be a...,160544,3683,49


In [17]:
sampled_df = pd.concat([resp_df_sample, resp_df_max], ignore_index=True)
sampled_df

,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_skills,job_summary,id,summary_len,skill_count
0,https://www.linkedin.com/jobs/view/software-ar...,2024-01-19 09:45:09.215838+00,t,t,f,Software Architect,Harris Computer,"Florida, United States",2024-01-13,Beverly Hills,United States,Director Technical,Mid senior,Onsite,"Product architecture, Engineering processes, C...",As Architect you will define the products’ hig...,340370,6407,33
1,https://www.linkedin.com/jobs/view/senior-syst...,2024-01-19 09:45:09.215838+00,t,t,f,Senior System Engineer,ExLabs,"Long Beach, CA",2024-01-13,Compton,United States,Maintenance Data Analyst,Mid senior,Onsite,"Systems engineering, Technical oversight, Requ...",Job Title: Sr. System Engineer\nCompany: ExLab...,640026,6308,32
2,https://www.linkedin.com/jobs/view/senior-data...,2024-01-20 11:15:59.804413+00,t,t,f,Senior Data Scientist,Workforce Connections,"St Louis, MO",2024-01-14,Belleville,United States,Chemist,Mid senior,Onsite,"Data Science, Data Analysis, Machine Learning,...","Location: Onsite, St. Louis MO 63146\nJob Resp...",1096469,3635,30
3,https://www.linkedin.com/jobs/view/distinguish...,2024-01-19 22:25:24.801979+00,t,t,f,Distinguished Applied Researcher,Jobs for Humanity,"Fort Worth, TX",2024-01-15,Fort Worth,United States,Director Research,Mid senior,Onsite,"AI, Applied Research, Machine Learning, Natura...",Company Description\nJobs for Humanity is part...,998553,4011,34
4,https://www.linkedin.com/jobs/view/systems-adm...,2024-01-21 01:06:09.90334+00,t,t,f,Systems Administrator,Leidos,"Colorado Springs, CO",2024-01-14,Cripple Creek,United States,Tier,Mid senior,Onsite,"System Administration, IT Operating Systems, C...",Description\nThe Leidos Digital Modernization ...,1189859,4117,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,https://uk.linkedin.com/jobs/view/workshop-con...,2024-01-19 09:45:09.215838+00,t,t,f,Workshop Controller,Activate Group Limited,"Bury St. Edmunds, England, United Kingdom",2024-01-13,Ipswich,United Kingdom,Controller,Mid senior,Onsite,"Workshop Control, Body Shop Management, HighQu...",Job title: Workshop Controller\nDepartment: Ac...,321651,4703,32
1496,https://www.linkedin.com/jobs/view/x-ray-techn...,2024-01-21 04:16:54.473867+00,t,t,f,X-Ray Technologist,Fast Pace Health,"Grant, AL",2024-01-17,Guntersville,United States,Ultrasound Technologist,Mid senior,Onsite,"XRay Technologist, HIPAA, Patient intake, Nurs...",Posting Title: X-Ray Technologist Overview:\nF...,1230454,5704,38
1497,https://www.linkedin.com/jobs/view/xfinity-ret...,2024-01-19 23:17:04.320124+00,t,t,f,Xfinity Retail Sales Consultant,XFINITY Store by Comcast,"East Brunswick, NJ",2024-01-14,Long Branch,United States,Sales-Service Promoter,Mid senior,Onsite,"Customer service, Product demonstration, Probl...",Job Summary\nResponsible for assisting and con...,159258,4839,32
1498,https://www.linkedin.com/jobs/view/yard-associ...,2024-01-19 09:45:09.215838+00,t,t,f,Yard Associate (SkillBridge),ARMOR Initiative,"Kansas City, MO",2024-01-13,Leavenworth,United States,Orderly,Associate,Onsite,"DoD SkillBridge Internship, Fulltime training,...",The ARMOR Initiative DoD SkillBridge Internshi...,774643,2277,30


In [18]:
sampled_df['id'].nunique()

1500

## Writing the sampled data

In [19]:
display(sampled_df.head(500))
display(sampled_df.head(1000).tail(500))
display(sampled_df.tail(500))

,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_skills,job_summary,id,summary_len,skill_count
0,https://www.linkedin.com/jobs/view/software-ar...,2024-01-19 09:45:09.215838+00,t,t,f,Software Architect,Harris Computer,"Florida, United States",2024-01-13,Beverly Hills,United States,Director Technical,Mid senior,Onsite,"Product architecture, Engineering processes, C...",As Architect you will define the products’ hig...,340370,6407,33
1,https://www.linkedin.com/jobs/view/senior-syst...,2024-01-19 09:45:09.215838+00,t,t,f,Senior System Engineer,ExLabs,"Long Beach, CA",2024-01-13,Compton,United States,Maintenance Data Analyst,Mid senior,Onsite,"Systems engineering, Technical oversight, Requ...",Job Title: Sr. System Engineer\nCompany: ExLab...,640026,6308,32
2,https://www.linkedin.com/jobs/view/senior-data...,2024-01-20 11:15:59.804413+00,t,t,f,Senior Data Scientist,Workforce Connections,"St Louis, MO",2024-01-14,Belleville,United States,Chemist,Mid senior,Onsite,"Data Science, Data Analysis, Machine Learning,...","Location: Onsite, St. Louis MO 63146\nJob Resp...",1096469,3635,30
3,https://www.linkedin.com/jobs/view/distinguish...,2024-01-19 22:25:24.801979+00,t,t,f,Distinguished Applied Researcher,Jobs for Humanity,"Fort Worth, TX",2024-01-15,Fort Worth,United States,Director Research,Mid senior,Onsite,"AI, Applied Research, Machine Learning, Natura...",Company Description\nJobs for Humanity is part...,998553,4011,34
4,https://www.linkedin.com/jobs/view/systems-adm...,2024-01-21 01:06:09.90334+00,t,t,f,Systems Administrator,Leidos,"Colorado Springs, CO",2024-01-14,Cripple Creek,United States,Tier,Mid senior,Onsite,"System Administration, IT Operating Systems, C...",Description\nThe Leidos Digital Modernization ...,1189859,4117,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,https://www.linkedin.com/jobs/view/arnp-at-fas...,2024-01-20 11:08:01.254837+00,t,t,f,ARNP,Fast Pace Health,"Prien, LA",2024-01-14,Lake Charles,United States,Nurse Practitioner,Mid senior,Onsite,"Family Nurse Practitioner, Teamwork, Communica...",Posting Title: Family Nurse Practitioner Overv...,955798,6118,31
496,https://www.linkedin.com/jobs/view/assistant-m...,2024-01-20 03:22:05.147606+00,t,t,f,ASSISTANT MANAGER,d'Lite Healthy On The Go,"Scottsdale, AZ",2024-01-15,Chandler,United States,Complaint Clerk,Mid senior,Onsite,"Restaurant management, Leadership, Customer se...","Location: Scottsdale, AZ\nEmployment Type: Ful...",13071,5375,43
497,https://ca.linkedin.com/jobs/view/academic-adv...,2024-01-19 09:45:09.215838+00,t,t,f,Academic Advisor,Mount Allison University,"Sackville, New Brunswick, Canada",2024-01-13,Sarnia-Clearwater,Canada,Foreign-Student Adviser,Mid senior,Onsite,"Academic advising, Transfer credit assessment,...",Position Description\nReporting to the Registr...,368698,6775,48
498,https://www.linkedin.com/jobs/view/account-exe...,2024-01-19 09:45:09.215838+00,t,t,f,Account Executive Digital Sales,Salem Media Group,"Alexandria, VA",2024-01-12,Dunkirk,United States,Account Executive,Mid senior,Onsite,"Sales, Account Executive, Media Strategist, Bu...",Overview\nSalem Media - Washington D.C.\noffer...,440051,2451,40


,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_skills,job_summary,id,summary_len,skill_count
500,https://www.linkedin.com/jobs/view/account-exe...,2024-01-19 09:45:09.215838+00,t,t,f,"Account Executive, Industrial Supply",Better Talent Advisors,"Stamford, CT",2024-01-12,Bridgeport,United States,Account Executive,Mid senior,Onsite,"Sales, Prospecting, Relationship Building, Bel...","With over 100 years in business, our family of...",359965,2141,36
501,https://www.linkedin.com/jobs/view/account-exe...,2024-01-19 09:45:09.215838+00,t,t,f,"Account Executive, Public Sector",Better Talent Advisors,"Rochester, NY",2024-01-12,Williamson,United States,Account Executive,Mid senior,Onsite,"Sales, Territory Management, Public Sector Sal...","With over 100 years in business, our family of...",192937,2220,34
502,https://www.linkedin.com/jobs/view/account-sup...,2024-01-19 12:35:07.03423+00,t,t,f,Account Supervisor,Publicis Digital Experience,"Birmingham, MI",2024-01-16,Macomb,United States,Supervisor Drawing,Mid senior,Onsite,"CRM, Automotive marketing, Subscriptions servi...",Hi there! We’re Razorfish. We’ve been leading ...,34636,6510,32
503,https://www.linkedin.com/jobs/view/accountant-...,2024-01-19 09:45:09.215838+00,t,t,f,Accountant II,Curaleaf,"Scottsdale, AZ",2024-01-12,Glendale,United States,Accounting Clerk,Mid senior,Onsite,"General ledger accounting, Financial analysis,...","Curaleaf Holdings, Inc. (CSE: CURA) (OTCQX: CU...",260902,6262,32
504,https://www.linkedin.com/jobs/view/accounting-...,2024-01-19 09:45:09.215838+00,t,t,f,Accounting Assistant,Jones Grove IT Recruiting,Charlotte Metro,2024-01-12,Hollywood,United States,Accounting Clerk,Mid senior,Onsite,"Accounting/Bookkeeping, Billing, Accounts Paya...",Accounting Assistant/Bookkeeper Direct Hire Ch...,477644,2737,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://www.linkedin.com/jobs/view/manufacturi...,2024-01-21 05:25:14.377269+00,t,t,f,Manufacturing Supervisor,Volm Companies Inc.,"Antigo, WI",2024-01-14,Wisconsin,United States,Superintendent Sales,Mid senior,Onsite,"ERP systems, Spreadsheets, Word processing, Co...",Job Title:\nManufacturing Supervisor C\nReport...,1287615,5863,53
996,https://www.linkedin.com/jobs/view/marketing-c...,2024-01-20 04:24:04.26509+00,t,t,f,Marketing Coordinator,Employee Owned Holdings,"Houston, TX",2024-01-14,Baytown,United States,Retirement Officer,Associate,Onsite,"eCommerce, Online shopping, Customer registrat...","Employee Owned Holdings, Inc. (EOHI) is a fami...",838752,5771,37
997,https://uk.linkedin.com/jobs/view/marketing-ex...,2024-01-19 09:45:09.215838+00,t,t,f,Marketing Executive,APT Travel Group,"London, England, United Kingdom",2024-01-13,Greater London,United Kingdom,Director Social,Mid senior,Onsite,"Marketing, B2C, Trade, TTL, Tactical Campaigns...","ABOUT US - Based In London, United Kingdom\nAP...",589429,5303,30
998,https://www.linkedin.com/jobs/view/marketing-s...,2024-01-19 09:45:09.215838+00,t,t,f,Marketing Specialist,VP Logistics,"Waukegan, IL",2024-01-13,North Chicago,United States,Guide,Associate,Onsite,"Marketing Strategies, Marketing Materials, Mar...",Job Summary:\nThe Marketing Specialist is resp...,235002,5127,30


,job_link,last_processed_time,got_summary,got_ner,is_being_worked,job_title,company,job_location,first_seen,search_city,search_country,search_position,job_level,job_type,job_skills,job_summary,id,summary_len,skill_count
1000,https://www.linkedin.com/jobs/view/master-sche...,2024-01-21 13:08:00.689444+00,t,t,f,Master Scheduler,Baltimore Aircoil Company,"Dayton, TN",2024-01-17,Tennessee,United States,Traffic Agent,Mid senior,Onsite,"Operations, Supply Chain, MRP systems, SAP ERP...","Baltimore Aircoil Company (BAC), an Amsted Ind...",866062,5864,77
1001,https://www.linkedin.com/jobs/view/master-tech...,2024-01-19 09:45:09.215838+00,t,t,f,Master Technician,A. O. Smith Corporation,"Lebanon, TN",2024-01-12,Watertown,United States,Assembler,Mid senior,Onsite,"Engineering, Product Development, Innovation, ...",Company / Location Information\nWater technolo...,531893,4514,32
1002,https://www.linkedin.com/jobs/view/material-ha...,2024-01-21 15:59:00.747387+00,t,t,f,Material Handler,IMI Critical Engineering,"Rancho Santa Margarita, CA",2024-01-16,Corona,United States,Route Supervisor,Associate,Onsite,"Forklift operation, Stacker operation, Metal c...","About IMI\nIMI plc, the specialist engineering...",927220,6292,80
1003,https://www.linkedin.com/jobs/view/materials-m...,2024-01-21 01:30:05.765203+00,t,t,f,Materials Manager,Pregis,"Bensenville, IL",2024-01-16,Elgin,United States,Manager Procurement Services,Mid senior,Onsite,"Leadership, Talent management, 5S program, Pro...",We're Not Just in the Packaging Business - We'...,1195068,5323,42
1004,https://www.linkedin.com/jobs/view/materials-p...,2024-01-19 13:06:03.847966+00,t,t,f,Materials Planner,Faraday Future,"Hanford, CA",2024-01-14,Selma,United States,Scheduler,Mid senior,Onsite,"SAP, MRP software, Excel, Word, PowerPoint, Ou...",The Company:\nFaraday Future (FF) is a Califor...,818707,5205,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,https://uk.linkedin.com/jobs/view/workshop-con...,2024-01-19 09:45:09.215838+00,t,t,f,Workshop Controller,Activate Group Limited,"Bury St. Edmunds, England, United Kingdom",2024-01-13,Ipswich,United Kingdom,Controller,Mid senior,Onsite,"Workshop Control, Body Shop Management, HighQu...",Job title: Workshop Controller\nDepartment: Ac...,321651,4703,32
1496,https://www.linkedin.com/jobs/view/x-ray-techn...,2024-01-21 04:16:54.473867+00,t,t,f,X-Ray Technologist,Fast Pace Health,"Grant, AL",2024-01-17,Guntersville,United States,Ultrasound Technologist,Mid senior,Onsite,"XRay Technologist, HIPAA, Patient intake, Nurs...",Posting Title: X-Ray Technologist Overview:\nF...,1230454,5704,38
1497,https://www.linkedin.com/jobs/view/xfinity-ret...,2024-01-19 23:17:04.320124+00,t,t,f,Xfinity Retail Sales Consultant,XFINITY Store by Comcast,"East Brunswick, NJ",2024-01-14,Long Branch,United States,Sales-Service Promoter,Mid senior,Onsite,"Customer service, Product demonstration, Probl...",Job Summary\nResponsible for assisting and con...,159258,4839,32
1498,https://www.linkedin.com/jobs/view/yard-associ...,2024-01-19 09:45:09.215838+00,t,t,f,Yard Associate (SkillBridge),ARMOR Initiative,"Kansas City, MO",2024-01-13,Leavenworth,United States,Orderly,Associate,Onsite,"DoD SkillBridge Internship, Fulltime training,...",The ARMOR Initiative DoD SkillBridge Internshi...,774643,2277,30


In [20]:
# Writing to 3 different sample files
sampled_df.head(500).to_csv(f"{path}/data/sampled_linkedin_jd_skills/sampled_jd_skills_set1.csv", index=False)
sampled_df.head(1000).tail(500).to_csv(f"{path}/data/sampled_linkedin_jd_skills/sampled_jd_skills_set2.csv", index=False)
sampled_df.tail(500).to_csv(f"{path}/data/sampled_linkedin_jd_skills/sampled_jd_skills_set3.csv", index=False)